In [25]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy
import seaborn as sns
from sklearn import linear_model
from sklearn.model_selection import train_test_split
import sklearn
from sklearn import tree
from IPython.display import Image
import pydotplus
import graphviz
from sklearn import ensemble
from sklearn.model_selection import cross_val_score
import time
import timeit
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Lasso
%matplotlib inline


In [2]:
df = pd.read_csv('survey.csv')
df = df.drop(['comments','Timestamp'], axis=1)
df.columns

Index(['Age', 'Gender', 'Country', 'state', 'self_employed', 'family_history',
       'treatment', 'work_interfere', 'no_employees', 'remote_work',
       'tech_company', 'benefits', 'care_options', 'wellness_program',
       'seek_help', 'anonymity', 'leave', 'mental_health_consequence',
       'phys_health_consequence', 'coworkers', 'supervisor',
       'mental_health_interview', 'phys_health_interview',
       'mental_vs_physical', 'obs_consequence'],
      dtype='object')

In [3]:
features = df.drop(['treatment'], axis=1)


In [4]:
df.replace(to_replace=['Yes', 'No'], value=[float(1),float(0)], inplace=True)

#df = pd.concat([df,treatmentBinary], axis=1)
#df['treatment_binary'] = pd.merge(df.No, df.Yes, on=df['No'])
#df = pd.concat([df,df.treatment_binary], axis=1)

In [5]:
features.replace(to_replace=['Yes', 'No'], value=[float(1),float(0)], inplace=True)

In [6]:
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)
df.head()

,Age,Gender,Country,state,self_employed,family_history,treatment,work_interfere,no_employees,remote_work,tech_company,benefits,care_options,wellness_program,seek_help,anonymity,leave,mental_health_consequence,phys_health_consequence,coworkers,supervisor,mental_health_interview,phys_health_interview,mental_vs_physical,obs_consequence
0,37,Female,United States,IL,NaN,0.0,1.0,Often,6-25,0.0,1.0,1,Not sure,0,1,1,Somewhat easy,0,0,Some of them,1,0,Maybe,1,0.0
1,44,M,United States,IN,NaN,0.0,0.0,Rarely,More than 1000,0.0,0.0,Don't know,0,Don't know,Don't know,Don't know,Don't know,Maybe,0,0,0,0,0,Don't know,0.0
2,32,Male,Canada,NaN,NaN,0.0,0.0,Rarely,6-25,0.0,1.0,0,0,0,0,Don't know,Somewhat difficult,0,0,1,1,1,1,0,0.0
3,31,Male,United Kingdom,NaN,NaN,1.0,1.0,Often,26-100,0.0,1.0,0,1,0,0,0,Somewhat difficult,1,1,Some of them,0,Maybe,Maybe,0,1.0
4,31,Male,United States,TX,NaN,0.0,0.0,Never,100-500,1.0,1.0,1,0,Don't know,Don't know,Don't know,Don't know,0,0,Some of them,1,1,1,Don't know,0.0


In [7]:
df_dummies = pd.get_dummies(df[['work_interfere', 'benefits', 'care_options', 'wellness_program',
       'seek_help', 'anonymity', 'leave', 'mental_health_consequence',
       'phys_health_consequence', 'coworkers', 'supervisor',
       'mental_health_interview', 'phys_health_interview',
       'mental_vs_physical', 'obs_consequence']])

In [8]:
df2 = pd.merge(df,df_dummies)

In [9]:
features['mental'] = df2['mental_vs_physical_0.0']

In [10]:
features['female'] = df.Gender.str.lower().apply(lambda x: 1 if x == 'female' else 0)

In [11]:
df_no = df.groupby('no_employees').treatment.mean().reset_index() 


In [12]:
df_no['mean_treatment_no']=df_no.treatment 
df_no.drop(['treatment'], axis=1, inplace= True)
df = pd.merge(df, df_no, on='no_employees') 

In [27]:
df.treatment.value_counts()

1.0    637
0.0    622
Name: treatment, dtype: int64

In [13]:
df.head()

,Age,Gender,Country,state,self_employed,family_history,treatment,work_interfere,no_employees,remote_work,tech_company,benefits,care_options,wellness_program,seek_help,anonymity,leave,mental_health_consequence,phys_health_consequence,coworkers,supervisor,mental_health_interview,phys_health_interview,mental_vs_physical,obs_consequence,mean_treatment_no
0,37,Female,United States,IL,NaN,0.0,1.0,Often,6-25,0.0,1.0,1,Not sure,0,1,1,Somewhat easy,0,0,Some of them,1,0,Maybe,1,0.0,0.441379
1,32,Male,Canada,NaN,NaN,0.0,0.0,Rarely,6-25,0.0,1.0,0,0,0,0,Don't know,Somewhat difficult,0,0,1,1,1,1,0,0.0,0.441379
2,33,Male,United States,TN,NaN,1.0,0.0,Sometimes,6-25,0.0,1.0,1,Not sure,0,Don't know,Don't know,Don't know,0,0,1,1,0,Maybe,Don't know,0.0,0.441379
3,31,Male,United States,OH,NaN,0.0,1.0,Sometimes,6-25,1.0,1.0,Don't know,0,0,0,Don't know,Don't know,0,0,Some of them,1,0,0,Don't know,0.0,0.441379
4,27,Male,Canada,NaN,NaN,0.0,0.0,Never,6-25,0.0,1.0,Don't know,Not sure,Don't know,Don't know,Don't know,Somewhat easy,0,0,Some of them,Some of them,Maybe,1,1,0.0,0.441379


In [14]:
features['mean_treatment_no'] = df['mean_treatment_no']

In [15]:
X = features[['female','family_history','Age','tech_company','remote_work','mental','mean_treatment_no']]
y = df.treatment

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [17]:
X_train.shape,y_train.shape

((881, 7), (881,))

In [18]:
print(timeit.timeit())
# Initialize and train our tree.
decision_tree = tree.DecisionTreeClassifier(
    criterion='entropy',
    max_features=5,
    max_depth=3,
    random_state = 1337
)
decision_tree.fit(X_train, y_train)

# Render our tree.
#dot_data = tree.export_graphviz(
 #   decision_tree, out_file=None,
    #feature_names=features,
    #class_names=[1, 0],
    #filled=True
 #   )
#graph = pydotplus.graph_from_dot_data(dot_data)
#Image(graph.create_png())

0.01747136196009918


DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=3,
            max_features=5, max_leaf_nodes=None, min_impurity_split=1e-07,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=1337,
            splitter='best')

In [19]:
decision_tree.feature_importances_

array([ 0.04280666,  0.0411032 ,  0.60242102,  0.        ,  0.1624111 ,
        0.        ,  0.15125802])

In [20]:
cross_val_score(decision_tree, X, y, cv=5).mean()

0.48776278286065222

In [21]:
print(timeit.timeit())
rfc = ensemble.RandomForestClassifier()
X = features[['female','family_history','Age','tech_company','remote_work']]
y = df.treatment

cross_val_score(rfc, X, y, cv=5).mean()

0.017361731508025002


0.53052611687636797

In [43]:
df.head()

,Age,Gender,Country,state,self_employed,family_history,work_interfere,no_employees,remote_work,tech_company,benefits,care_options,wellness_program,seek_help,anonymity,leave,mental_health_consequence,phys_health_consequence,coworkers,supervisor,mental_health_interview,phys_health_interview,mental_vs_physical,obs_consequence,mean_treatment_no
0,37,Female,United States,IL,NaN,0.0,Often,6-25,0.0,1.0,1,Not sure,0,1,1,Somewhat easy,0,0,Some of them,1,0,Maybe,1,0.0,0.441379
1,32,Male,Canada,NaN,NaN,0.0,Rarely,6-25,0.0,1.0,0,0,0,0,Don't know,Somewhat difficult,0,0,1,1,1,1,0,0.0,0.441379
2,33,Male,United States,TN,NaN,1.0,Sometimes,6-25,0.0,1.0,1,Not sure,0,Don't know,Don't know,Don't know,0,0,1,1,0,Maybe,Don't know,0.0,0.441379
3,31,Male,United States,OH,NaN,0.0,Sometimes,6-25,1.0,1.0,Don't know,0,0,0,Don't know,Don't know,0,0,Some of them,1,0,0,Don't know,0.0,0.441379
4,27,Male,Canada,NaN,NaN,0.0,Never,6-25,0.0,1.0,Don't know,Not sure,Don't know,Don't know,Don't know,Somewhat easy,0,0,Some of them,Some of them,Maybe,1,1,0.0,0.441379


In [44]:
X = df


In [37]:
type(X)


NoneType

In [53]:
X_train, X_test, y_train, y_test = train_test_split(X_dummies, y, test_size=0.3)

In [49]:
X_dummies = pd.get_dummies(df)

In [52]:
X_dummies.fillna(value=0)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


In [55]:
X_dummies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1259 entries, 0 to 1258
Columns: 200 entries, Age to mental_vs_physical_Don't know
dtypes: float64(6), int64(1), uint8(193)
memory usage: 356.0 KB


In [22]:
# Declare a logistic regression classifier.
# Parameter regularization coefficient C described above.
lr = LogisticRegression(C=1e9)


# Fit the model.
fit = lr.fit(X_train, y_train)

# Display.
print('Coefficients')
print(fit.coef_)
print(fit.intercept_)
pred_y_sklearn = lr.predict(X_test)

print('\n Accuracy by treatment status')
print(pd.crosstab(pred_y_sklearn, y_test))

print('\n Percentage accuracy')
print(lr.score(X_test, y_test))

Coefficients
[[  6.40308317e-18   1.92081291e-17  -1.12029643e-10  -3.41508926e-17
    1.70748882e-17  -1.06718039e-17   2.83468529e-18]]
[ -4.26983922e-18]

 Accuracy by treatment status
treatment  0.0  1.0
row_0              
0.0        179  197
1.0          1    1

 Percentage accuracy
0.47619047619


In [23]:
# Declare a logistic regression classifier.
# Parameter regularization coefficient C described above.
lr = LogisticRegression(penalty='l1',C=1e9)


# Fit the model.
fit = lr.fit(X_train, y_train)

# Display.
print('Coefficients')
print(fit.coef_)
print(fit.intercept_)
pred_y_sklearn = lr.predict(X_test)

print('\n Accuracy by treatment status')
print(pd.crosstab(pred_y_sklearn, y_test))

print('\n Percentage accuracy')
print(lr.score(X_test, y_test))

Coefficients
[[  1.71912001e-02   7.41373939e-02  -1.12021250e-10  -2.22546161e-01
    9.95987609e-02  -5.70325310e-02   2.21150392e-01]]
[ 0.01504946]

 Accuracy by treatment status
treatment  0.0  1.0
row_0              
0.0        114  124
1.0         66   74

 Percentage accuracy
0.497354497354


In [24]:
# Declare a logistic regression classifier.
# Parameter regularization coefficient C described above.
lr = LogisticRegression(penalty='l2',C=1e9)


# Fit the model.
fit = lr.fit(X_train, y_train)

# Display.
print('Coefficients')
print(fit.coef_)
print(fit.intercept_)
pred_y_sklearn = lr.predict(X_test)

print('\n Accuracy by treatment status')
print(pd.crosstab(pred_y_sklearn, y_test))

print('\n Percentage accuracy')
print(lr.score(X_test, y_test))

Coefficients
[[  6.40308317e-18   1.92081291e-17  -1.12029643e-10  -3.41508926e-17
    1.70748882e-17  -1.06718039e-17   2.83468529e-18]]
[ -4.26983922e-18]

 Accuracy by treatment status
treatment  0.0  1.0
row_0              
0.0        179  197
1.0          1    1

 Percentage accuracy
0.47619047619


In [54]:
lasso = linear_model.Lasso()
fit = lasso.fit(X_train, y_train)

# Display.
print('Coefficients')
print(fit.coef_)
print(fit.intercept_)
pred_y_sklearn = lasso.predict(X_test)

print('\n Accuracy by treatment status')
print(pd.crosstab(pred_y_sklearn, y_test))

print('\n Percentage accuracy')
print(lasso.score(X_test, y_test))

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').